In [ ]:
import tensorflow as tf

tf.test.gpu_device_name()

'/device:GPU:0'

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/My Drive/Kaggle"

# /content/gdrive/My Drive/Kaggle is the path where kaggle.json is present in the Google Drive

In [ ]:
#changing the working directory

%cd /content/gdrive/My Drive/Kaggle


/content/gdrive/My Drive/Kaggle


In [ ]:
% pwd

#Check the present working directory using pwd command

'/content/gdrive/My Drive/Kaggle'

In [ ]:
!ls


basic_benchmark.csv.zip      saved_models	train.csv
kaggle.json		     submission.csv	train_embed_df.csv
prior_benchmark.csv.zip      test.csv.zip	train_embed_df.pkl
sampleSubmission.csv	     test_embed_df.csv	train-sample.csv.zip
sampleSubmission.csv.zip     test_embed_df.pkl	train.tsv
sampleSubmission_v2.csv.zip  test.tsv		train_v2.csv


In [ ]:
!kaggle competitions files stumbleupon

# To download a specific file I normally use:
# !kaggle competitions download home-depot-product-search-relevance -f train.csv.zip

name                   size  creationDate         
--------------------  -----  -------------------  
test.tsv                9MB  2018-04-30 03:45:41  
train.tsv              21MB  2018-04-30 03:45:41  
raw_content.zip       157MB  2018-04-30 03:45:41  
sampleSubmission.csv   22KB  2018-04-30 03:45:41  


In [ ]:
!kaggle competitions download stumbleupon -f train.tsv

100% 8.00M/8.02M [00:00<00:00, 82.1MB/s]
100% 8.02M/8.02M [00:00<00:00, 73.9MB/s]


In [ ]:
! ls

basic_benchmark.csv.zip      submission.csv	 train_embed_df.pkl
kaggle.json		     test.csv.zip	 train-sample.csv.zip
prior_benchmark.csv.zip      test_embed_df.csv	 train.tsv
sampleSubmission.csv	     test_embed_df.pkl	 train.tsv.zip
sampleSubmission.csv.zip     test.tsv		 train_v2.csv
sampleSubmission_v2.csv.zip  train.csv
saved_models		     train_embed_df.csv


In [ ]:
#unzipping the zip files and deleting the zip files

!unzip -o \train.tsv.zip  && rm train.tsv.zip

Archive:  train.tsv.zip
  inflating: train.tsv               


In [ ]:
import pandas as pd

df = pd.read_csv("train.tsv", sep = '\t')
df.head()

,url,urlid,boilerplate,alchemy_category,alchemy_category_score,avglinksize,commonlinkratio_1,commonlinkratio_2,commonlinkratio_3,commonlinkratio_4,compression_ratio,embed_ratio,framebased,frameTagRatio,hasDomainLink,html_ratio,image_ratio,is_news,lengthyLinkDomain,linkwordscore,news_front_page,non_markup_alphanum_characters,numberOfLinks,numwords_in_url,parametrizedLinkRatio,spelling_errors_ratio,label
0,http://www.bloomberg.com/news/2010-12-23/ibm-p...,4042,"{""title"":""IBM Sees Holographic Calls Air Breat...",business,0.789131,2.055556,0.676471,0.205882,0.047059,0.023529,0.443783,0.0,0,0.090774,0,0.245831,0.003883,1,1,24,0,5424,170,8,0.152941,0.079130,0
1,http://www.popsci.com/technology/article/2012-...,8471,"{""title"":""The Fully Electronic Futuristic Star...",recreation,0.574147,3.677966,0.508021,0.288770,0.213904,0.144385,0.468649,0.0,0,0.098707,0,0.203490,0.088652,1,1,40,0,4973,187,9,0.181818,0.125448,1
2,http://www.menshealth.com/health/flu-fighting-...,1164,"{""title"":""Fruits that Fight the Flu fruits tha...",health,0.996526,2.382883,0.562016,0.321705,0.120155,0.042636,0.525448,0.0,0,0.072448,0,0.226402,0.120536,1,1,55,0,2240,258,11,0.166667,0.057613,1
3,http://www.dumblittleman.com/2007/12/10-foolpr...,6684,"{""title"":""10 Foolproof Tips for Better Sleep ""...",health,0.801248,1.543103,0.400000,0.100000,0.016667,0.000000,0.480725,0.0,0,0.095861,0,0.265656,0.035343,1,0,24,0,2737,120,5,0.041667,0.100858,1
4,http://bleacherreport.com/articles/1205138-the...,9006,"{""title"":""The 50 Coolest Jerseys You Didn t Kn...",sports,0.719157,2.676471,0.500000,0.222222,0.123457,0.043210,0.446143,0.0,0,0.024908,0,0.228887,0.050473,1,1,14,0,12032,162,10,0.098765,0.082569,0


In [ ]:
df.shape

entry = df['boilerplate'][0]

print(entry)

# test = pd.read_csv("test.tsv", sep = '\t')
# test.head()

# test.shape

{"title":"IBM Sees Holographic Calls Air Breathing Batteries ibm sees holographic calls, air-breathing batteries","body":"A sign stands outside the International Business Machines Corp IBM Almaden Research Center campus in San Jose California Photographer Tony Avelar Bloomberg Buildings stand at the International Business Machines Corp IBM Almaden Research Center campus in the Santa Teresa Hills of San Jose California Photographer Tony Avelar Bloomberg By 2015 your mobile phone will project a 3 D image of anyone who calls and your laptop will be powered by kinetic energy At least that s what International Business Machines Corp sees in its crystal ball The predictions are part of an annual tradition for the Armonk New York based company which surveys its 3 000 researchers to find five ideas expected to take root in the next five years IBM the world s largest provider of computer services looks to Silicon Valley for input gleaning many ideas from its Almaden research center in San Jose 

In [ ]:
# Below code is from link: https://towardsdatascience.com/multi-label-multi-class-text-classification-with-bert-transformer-and-keras-c6355eccb63a

# Load libraries
# Load what you need from tensorflow.keras

import tensorflow as tf
from tensorflow.keras.layers import Input, Dropout, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import roc_auc_score
from tensorflow import keras
from sklearn.metrics import roc_auc_score

# Add pandas for data import and sklearn because you always need sklearn

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [ ]:
# Select required columns
data = df[['boilerplate', 'label']]



In [ ]:
# Remove the row if any of the column data is missing
data = data.dropna()

# Remove rows where label is present only ones (cant be split)
# data = data.groupby('OpenStatus').filter(lambda x: len(x) > 1)



In [ ]:
print(type(data['label'][0]))

<class 'numpy.int64'>


In [ ]:
!pip install -U sentence-transformers

from sentence_transformers import SentenceTransformer

     |████████████████████████████████| 78 kB 3.2 MB/s 
     |████████████████████████████████| 3.1 MB 11.5 MB/s 
     |████████████████████████████████| 3.3 MB 31.4 MB/s 
     |████████████████████████████████| 1.2 MB 36.2 MB/s 
     |████████████████████████████████| 56 kB 5.2 MB/s 
     |████████████████████████████████| 895 kB 37.7 MB/s 
     |████████████████████████████████| 596 kB 45.7 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.1.0-py3-none-any.whl size=121000 sha256=cc77d1e8ac58959f5d7ff168ec0f96cec111685011630b17e17c7b274c6fa37a
  Stored in directory: /root/.cache/pip/wheels/90/f0/bb/ed1add84da70092ea526466eadc2bfb197c4bcb8d4fa5f7bad
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
model = SentenceTransformer('all-mpnet-base-v2')
boilerplate = model.encode(data['boilerplate'].to_numpy())

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.1k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/571 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/39.3k [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/363 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.1k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
print(type(boilerplate))
print(boilerplate)
print('------')
print(len(boilerplate[0]))
print(type(boilerplate[0]))
print(boilerplate[0])
print('------')
print(type(boilerplate[0][0]))
print(boilerplate[0][0])

<class 'numpy.ndarray'>
[[ 0.07850941  0.05262044 -0.02368153 ... -0.01866526 -0.03950941
  -0.04052584]
 [-0.05454525  0.00235558  0.02027129 ...  0.03061437 -0.04803398
   0.02790574]
 [ 0.01705634  0.02868466 -0.03189581 ...  0.00178062  0.01224712
  -0.0770412 ]
 ...
 [ 0.01113353  0.03346698  0.01648079 ...  0.00265327 -0.02813832
  -0.0060337 ]
 [ 0.07747136  0.02302336 -0.03306821 ... -0.03418817 -0.06583659
   0.01368981]
 [ 0.04951881  0.05973886 -0.01139303 ... -0.02941196 -0.03759442
   0.01034244]]
------
768
<class 'numpy.ndarray'>
[ 7.85094053e-02  5.26204370e-02 -2.36815251e-02 -1.21802166e-02
 -4.93502198e-03 -2.50766277e-02  2.66505815e-02  1.93444118e-02
 -2.50174142e-02 -9.34470631e-03  3.82753685e-02  3.96296978e-02
  2.20900271e-02  8.94398242e-02  2.62531769e-02 -1.58181153e-02
  4.39353250e-02  5.67080639e-02 -4.41321954e-02 -5.50679979e-04
 -1.54180238e-02  1.56977754e-02 -6.10001502e-04  1.59468334e-02
  6.34090463e-03  7.59655796e-03 -7.16692128e-04  2.9991015

In [ ]:
!ls

basic_benchmark.csv.zip      saved_models	train.csv
kaggle.json		     submission.csv	train_embed_df.csv
prior_benchmark.csv.zip      test.csv.zip	train_embed_df.pkl
sampleSubmission.csv	     test_embed_df.csv	train-sample.csv.zip
sampleSubmission.csv.zip     test_embed_df.pkl	train.tsv
sampleSubmission_v2.csv.zip  test.tsv		train_v2.csv


In [ ]:
#unzipping the zip files and deleting the zip files

!unzip -o \test.tsv.zip  && rm test.tsv.zip

unzip:  cannot find or open test.tsv.zip, test.tsv.zip.zip or test.tsv.zip.ZIP.


In [ ]:
import pandas as pd

test = pd.read_csv("test.tsv", sep = '\t')
test.head()

,url,urlid,boilerplate,alchemy_category,alchemy_category_score,avglinksize,commonlinkratio_1,commonlinkratio_2,commonlinkratio_3,commonlinkratio_4,compression_ratio,embed_ratio,framebased,frameTagRatio,hasDomainLink,html_ratio,image_ratio,is_news,lengthyLinkDomain,linkwordscore,news_front_page,non_markup_alphanum_characters,numberOfLinks,numwords_in_url,parametrizedLinkRatio,spelling_errors_ratio
0,http://www.lynnskitchenadventures.com/2009/04/...,5865,"{""title"":""Homemade Enchilada Sauce Lynn s Kitc...",recreation,0.443906,2.558140,0.389706,0.257353,0.044118,0.022059,0.489572,0.0,0,0.067143,0,0.230285,0.199438,1,1,15,0,5643,136,3,0.242647,0.080597
1,http://lolpics.se/18552-stun-grenade-ar,782,"{""title"":""lolpics Stun grenade ar "",""body"":"" f...",culture_politics,0.135844,3.771429,0.461538,0.205128,0.051282,0.000000,0.782051,0.0,0,0.042857,0,0.365962,0.080000,?,1,62,0,382,39,2,0.128205,0.176471
2,http://www.xcelerationfitness.com/treadmills.html,6962,"{""title"":""Treadmills "",""body"":"" treadmills, st...",?,?,2.269565,0.495726,0.384615,0.170940,0.170940,1.250000,0.0,0,0.058824,0,0.161901,10.000000,?,1,42,0,2420,117,1,0.581197,0.125000
3,http://www.bloomberg.com/news/2012-02-06/syria...,7640,"{""title"":""Father s Tactics Used by Assad to Cr...",culture_politics,0.90259,2.523490,0.705502,0.346278,0.122977,0.090615,0.449366,0.0,0,0.058081,0,0.146593,0.005964,1,1,41,0,5559,309,10,0.038835,0.063126
4,http://www.wired.com/gadgetlab/2011/12/stem-tu...,3589,"{""title"":""Stem Turns Lemons and Limes Into Jui...",science_technology,0.486363,1.848000,0.470968,0.161290,0.032258,0.000000,0.453757,0.0,0,0.093023,0,0.244141,0.035714,1,0,34,0,2209,155,10,0.096774,0.065341


In [ ]:
test_data = test[['urlid', 'boilerplate']]
test_data.head()

,urlid,boilerplate
0,5865,"{""title"":""Homemade Enchilada Sauce Lynn s Kitc..."
1,782,"{""title"":""lolpics Stun grenade ar "",""body"":"" f..."
2,6962,"{""title"":""Treadmills "",""body"":"" treadmills, st..."
3,7640,"{""title"":""Father s Tactics Used by Assad to Cr..."
4,3589,"{""title"":""Stem Turns Lemons and Limes Into Jui..."


In [ ]:
test_boilerplate = model.encode(test_data['boilerplate'].to_numpy())
print(type(test_boilerplate))


In [ ]:
print(test_boilerplate)
print('------')
print(type(test_boilerplate[0]))
print('------')
print(type(test_boilerplate[0][0]))

In [ ]:
data['boilerplate_embed'] = boilerplate.tolist()
print(type(data['boilerplate_embed']))
data.head()

In [ ]:
# Set your model output as categorical and and save in new label column
data['binary_label'] = pd.Categorical(data['label'])

# Transform your output to numeric
data['label'] = data['binary_label'].cat.codes

In [ ]:
print(type(data['binary_label'][0]))

In [ ]:
print(type(data['label'][0]))

In [ ]:
data['label'] = data['label'].to_numpy(dtype = 'float32')
print(type(data['label'][0]))

In [ ]:
data.head()

In [ ]:
print(data.loc[:,'binary_label'])

In [ ]:
print(type(data['label'][0]))

In [ ]:
print(type(boilerplate))

In [ ]:
# ## Code to save embeddings in a file using PICKLE so that i dont have to run them again

# train_embed_df = pd.DataFrame(columns = ['train_embed','label'])
# train_embed_df['train_embed'] = boilerplate.tolist()
# train_embed_df['label'] = data['label']
# train_embed_df.to_pickle("train_embed_df.pkl")
# # train_embed_df.to_csv("train_embed_df.csv")

# test_embed_df = pd.DataFrame(columns = ['test_embed','label'])
# test_embed_df['test_embed'] = test_boilerplate.tolist()
# test_embed_df.to_pickle("test_embed_df.pkl")
# # test_embed_df.to_csv("test_embed_df.csv")

In [ ]:
x = boilerplate
print(type(x))
print(len(x))
print(len(x[0]))
print(len(x[0]))

In [ ]:
import numpy as np 

y = np.array(data['label'])
print(y)
print(type(y))
print(len(y))

In [ ]:
# import pandas as pd
# import numpy as np
# train_embed_df = pd.read_pickle("train_embed_df.pkl")
# train_embed_df.head()

In [ ]:
# x = (train_embed_df['train_embed']).to_numpy()
# print(x)
# print(type(x))
# print(len(x))
# print(x[0])
# print(type(x[0]))
# print(len(x[0]))
# print(type(x[0][0]))
# print(x[0][0])

In [ ]:
# y = (train_embed_df['label']).to_numpy()
# print(y)
# print(type(y))
# print(y[0])
# print(type(y[0]))

In [ ]:
  print('x.value_counts() || y.value_counts())')
  (unique, counts) = np.unique(x, return_counts=True)
  print(f'unique {unique} counts {counts}')
  print('-----')
  (unique, counts) = np.unique(y, return_counts=True)
  print(f'unique {unique} counts {counts}')
  print('-----')

In [ ]:
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score

# # Create dataset
# x , y = make_classification(n_samples=400, n_features=20, n_informative=15, n_redundant=5, n_classes = 4, random_state=1)

# print(x)
# print(len(x))
# print(y)
# print(len(y))

# Prepare the cross vaidation procedure
# cv = StratifiedKFold(n_splits = 3, random_state = 1, shuffle = True)

In [ ]:
# Binary Classification with Sonar Dataset: Baseline

import pandas as pd
from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold, StratifiedKFold
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
import os

# kf = KFold(n_splits = 10)
skf = StratifiedKFold(n_splits = 7, random_state = 1, shuffle = True)

# We also need to save the best model in each fold
def get_model_name(k):
  return 'model_' + str(k) + '.h5'

train_accuracy = []
train_loss = []
train_auroc = []
history_list = []

save_dir = '/content/gdrive/My Drive/Kaggle/saved_models/'
fold_var = 1

for train_index, val_index in skf.split(np.zeros(len(x)), y):

  print('Fold variable ' + str(fold_var))

  # print('train_index || len(train_index) || val_index || len(val_index)')
  # print(train_index)
  # print(len(train_index))
  # print('-----')
  # print(val_index)
  # print(len(val_index))
  # print('-----')

  x_train_fold, y_train_fold = x[train_index] , y[train_index]
  x_val_fold, y_val_fold = x[val_index] , y[val_index]

  # print('x_train_fold || len(x_train_fold) || y_train_fold || len(y_train_fold || x_val_fold || len(x_val_fold) || y_val_fold || len(y_val_fold)')
  # print(x_train_fold)
  # print(len(x_train_fold))
  # print('-----')
  # print(y_train_fold)
  # print(len(y_train_fold))
  # print('-----')
  # print(x_val_fold)
  # print(len(x_val_fold))
  # print('-----')
  # print(x_val_fold)
  # print(len(x_val_fold))
  print('-----')
  print(y_val_fold)
  print(len(y_val_fold))
  print('-----')


  # print('y_train_fold.value_counts() || y_val_fold.value_counts())')
  # (unique, counts) = np.unique(x_val_fold, return_counts=True)
  # print(f'unique {unique} counts {counts}')
  # print('-----')
  # (unique, counts) = np.unique(y_val_fold, return_counts=True)
  # print(f'unique {unique} counts {counts}')
  # print('-----')

  # print('Validation set bifurcation')
  # print(type(y_train_fold))
  # import math
  # a = y_train_fold[-math.floor(0.15*len(y_train_fold)):]
  # print('-----')
  # (unique, counts) = np.unique(a, return_counts=True)
  # print(f'unique {unique} counts {counts}')

  # Create model

  def create_model():

    # n_features = 1
    # Define metrics

#     from sklearn.metrics import roc_auc_score
# def auc_score(y_true, y_pred):
#     if len(np.unique(y_true[:,1])) == 1:
#         return 0.5
#     else:
#         return roc_auc_score(y_true, y_pred)
# def auc(y_true, y_pred):
#     return tf.py_func(auc1, (y_true, y_pred), tf.double)

    def auroc(y_true, y_pred):
      print('y_true')
      print(y_true)
      print('y_pred')
      print(y_pred)
      return tf.numpy_function(roc_auc_score, (y_true, y_pred), tf.double)

    model = Sequential()
    model.add(Dense(60, input_dim = len(x[0]), activation = 'relu'))
    model.add(Dense(1, activation = 'sigmoid'))

    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics = ['accuracy', auroc])
    return model

  model_deep = create_model()

  # Create callbacks
  checkpoint_filepath = save_dir + get_model_name(fold_var)
  checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath = checkpoint_filepath, monitor = 'val_auroc', verbose = 1, save_best_only = True, mode = 'max')
  callbacks_list = [checkpoint]
  # There can be other callbacks, but just showing one because it involves the model name
	# This saves the best model

  # Fit the model
  history = model_deep.fit(x_train_fold, y_train_fold, callbacks = callbacks_list, epochs = 10, batch_size = 128, shuffle=True, verbose = 1, validation_data = (x_val_fold, y_val_fold))

  # Plot history - write code

  # Load best model to evaluate the performance of the model

  model_deep.load_weights('/content/gdrive/My Drive/Kaggle/saved_models/model_' + str(fold_var) + '.h5')
  
  # Evaluate the model

  results = model_deep.evaluate(x_val_fold, y_val_fold, batch_size = 128, verbose=1)

  print(f'Model_deep.metrics {model_deep.metrics_names} {results}')
  results = dict(zip(model_deep.metrics_names, results))

  train_accuracy.append(results['accuracy'])
  train_loss.append(results['loss'])
  train_auroc.append(results['auroc'])
  history_list.append(history.history)

  tf.keras.backend.clear_session()

  fold_var += 1


In [ ]:
print(train_loss)
print(train_accuracy)
print(train_auroc)

In [ ]:
# Gives the name of what model.evaluate outputs below
print(model_deep.metrics_names)

In [ ]:
print(history.history)

In [ ]:
import matplotlib.pyplot as plt
# visualizing losses and accuracy
train_loss = history.history['loss']
val_loss   = history.history['val_loss']
train_acc  = history.history['accuracy']
val_acc    = history.history['val_accuracy']
train_auroc  = history.history['accuracy']
val_auroc    = history.history['val_accuracy']
xc         = range(10)

plt.figure()
plt.plot(xc, train_loss)



In [ ]:
plt.plot(xc, val_loss)

In [ ]:
plt.plot(xc, train_acc)
plt.plot(xc, val_acc)


In [ ]:
plt.plot(xc, train_auroc)
plt.plot(xc, val_auroc)

In [ ]:
print(type(test_boilerplate))

In [ ]:
predictions = model_deep.predict(test_boilerplate)

In [ ]:
print(predictions)

In [ ]:
test_data.head()

In [ ]:
test_data['label'] = predictions
test_data.head()

In [ ]:
submission = pd.DataFrame(columns = ['urlid', 'label'])
submission.head()

In [ ]:
submission['urlid'] = test_data['urlid']
submission['label'] = test_data['label']
submission.head()

In [ ]:
submission.to_csv("submission.csv")

In [ ]:
!ls

In [ ]:
# from numpy import mean
# from numpy import std
# from sklearn.datasets import make_classification
# from sklearn.model_selection import StratifiedKFold
# from sklearn.model_selection import cross_val_score
# # Create dataset
# x , y = make_classification(n_samples=400, n_features=2, n_informative=2, n_redundant=0, random_state=1)

# print(x)
# print(len(x))
# print(y)
# print(len(y))


In [ ]:


# # We also need to save the best model in each fold
# def get_model_name(k):
#   return 'model_' + str(k) + '.h5'

# validation_accuracy = []
# validation_loss = []
# validation_auroc = []

# save_dir = '/content/gdrive/My Drive/Kaggle/saved_models/'
# fold_var = 1

# for train_index, val_index in skf.split(np.zeros(len(x)), y):
#   # print(train_index)
#   # print(len(train_index))
#   # print('-----')
#   # print(val_index)
#   # print(len(val_index))
#   x_train_fold, y_train_fold = x[train_index] , y[train_index]
#   x_val_fold, y_val_fold = x[val_index] , y[val_index]

#   # Create model

#   def create_model():

#     # n_features = 1
#     # Define metrics
#     def auroc(y_true, y_pred):
#         return tf.numpy_function(roc_auc_score, (y_true, y_pred), tf.double)

#     model = Sequential()
#     model.add(Dense(60, input_dim = len(x[0]), activation = 'relu'))
#     model.add(Dense(1, activation = 'sigmoid'))

#     # Compile model
#     model.compile(loss='binary_crossentropy', optimizer='adam', metrics = ['accuracy', auroc])
#     return model

#   model_deep = create_model()

#   # Create callbacks
#   checkpoint_filepath = save_dir + get_model_name(fold_var)
#   checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath = checkpoint_filepath, monitor = 'auroc', verbose = 1, save_best_only = True, mode = 'max')
#   callbacks_list = [checkpoint]
#   # There can be other callbacks, but just showing one because it involves the model name
# 	# This saves the best model

#   # Fit the model
#   history = model_deep.fit(x_train_fold, y_train_fold, callbacks = callbacks_list, epochs = 1, batch_size = 32, verbose = 1, validation_split = 0.15)

#   # Plot history - write code


#   # # Create a basic model instance
#   # model_deep = create_model()

#   # # Evaluate the model
#   # loss, acc = model_deep.evaluate(x_val_fold, y_val_fold, verbose=1)
#   # print("Untrained model, accuracy: {:5.2f}%".format(100 * acc))

#   # Load best model to evaluate the performance of the model

#   model_deep.load_weights('/content/gdrive/My Drive/Kaggle/saved_models/model_' + str(fold_var) + '.h5')
#   results = model_deep.evaluate(x_val_fold, y_val_fold, verbose=1)

#   print(f'Model_deep.metrics {model_deep.metrics_names} {results}')
#   results = dict(zip(model_deep.metrics_names, results))

#   validation_accuracy.append(results['accuracy'])
#   validation_loss.append(results['loss'])
#   validation_auroc.append(results['auroc'])

#   tf.keras.backend.clear_session()

#   fold_var += 1
#   print('Fold variable ' + str(fold_var))
